# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Elvis David Pachacama Cabezas  <br>
Url: https://github.com/ElvisDavis/maestria-algoritmos/blob/main/Seminario_Algoritmos.ipynb<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema: Se precisa coordinar el doblaje de una pelicula. Los actores del doblaje deben coincidir en la tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el mejor posible. Los datos son:<br>
**Número de actores: 10**<br>
**Número de tomas: 30**<br>
**Actores/Toma: https://bit.ly/36D8IuK**
* 1 indica que actor participa en la toma
* 0 en caso contrario


(*) La respuesta es obligatoria





                                        

In [1]:
#Importamos la librerias necesarias para leer el csv
import pandas as pd
import numpy as np
import random


In [2]:
#leemos el archivo

archivo = "Datos_problema_doblaje.csv"

#Cargamos el archivo csv
df= pd.read_csv(archivo)

#Se establece la primera fila como encabezado real
df.columns = df.iloc[0]
df = df.drop(index=0).reset_index(drop=True)

# Renombramos la primera comñlumna como Toma

df = df.rename(columns={df.columns[0]: "Toma"})
# Eliminamos columnas no necesarias
df = df.drop(columns=[col for col in df.columns if pd.isna(col) or col =='Total'])
#Eliminamos fila resumen
df = df[~df["Toma"].isin(["TOTAL"])]

#Eliminamos filas Nan en toma
df = df.dropna(subset=["Toma"])
#Convertir a entero
df["Toma"] = df["Toma"].astype(int)
# Tomas ordenadas
tomas_ordenadas = sorted(df["Toma"].tolist())
df.iloc[:,1:]= df.iloc[:,1:].apply(pd.to_numeric)

#Preparamos la lista (toma,[actores])
tomas_actores=[]
for _, row in df.iterrows():
    actores_en_toma = []
    for actor, valor in row.iloc[1:].items():
        if valor == 1.0:
            #convertir el nombre del actor en entero
            actores_en_toma.append(int(float(actor)))
    tomas_actores.append((row["Toma"], actores_en_toma))

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




In [3]:
df

,Toma,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
6,7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [4]:
sessions =np.array(df)

Respuesta

In [5]:
def scenes_organization_gh(sessions, skip_scenes):
    '''Inicializa:
    Crea una lista de todos las escenas no asignadas
    Crea una lista vacia para cada day para asignar la scenes.
    '''
    num_scenes = sessions.shape[0]
    unassigned = set(range(num_scenes))
    days = []
    ''' Mientras las escenas no esten asignadas:
    Seleccionamos una escena que no este asignada cmo la semilla para un nuevo día
    Para este, día intenta agregar hasta 5 escenas no asignadas adicionales que tengan la
    mayor coincidencia de actores con el grupo actual (es decir, que al agregarlas aumenten
    lo menos posible el conjunto de actores únicos. Asigna estas escenas al día actual y elimpinalas de
    la lista de escenas no asignadas'''
    
    while unassigned:
        #obtnemos una escena cualquiera de la lista no asignada
        seed_scene = random.choice(list(unassigned))
        day_scenes= [seed_scene]
        unassigned.remove(seed_scene)

        #Obtenmos el set de actores presentes en la primera escena
        actors_in_day= set(np.where(sessions[day_scenes[0]]==1)[0])

        #Tratamos de añadir más de 5 escenas para este día
        for _ in range(5):
            best_scene = None
            best_increase = None
            #Iteramos sobre las escenas no asignadas
            for scene in unassigned:
                #Caculamos el aumento en el número de actores únicos si se añade a esta escena
                #actors_in_day es el conjunto de datos de actores ya asignados a este día.
                #actors_in_scene es el conjunto de datos de actores en la escena que se está analizando actualmente
                actors_in_scene= set(np.where(sessions[scene] == 1)[0])
                #esto fue pensado para omitir escenas que solo tuvieran actores no presentes actualmente,
                #pero termino generando peores resultados
                
                if skip_scenes and actors_in_scene.difference(actors_in_day) == set():
                    continue #Saltamos si no se repiten actores nuevos
                increase = len(actors_in_day | actors_in_scene) - len(actors_in_day)
                if best_scene is None or increase < best_increase:
                    best_scene = scene
                    best_increase = increase
            if best_scene is not None:
                day_scenes.append(best_scene)
                actors_in_day |= set(np.where(sessions[best_scene] == 1)[0])#actualiza el cojunto de datos
                unassigned.remove(best_scene)
            else:
                break
        days.append(day_scenes)
    return days



In [6]:
def calc_cost(days, sessions):
    #Para cada día, contamos los actores únicos presentes ( es decir, cualquier actor que tenga un valor de 1 en alguna escena asignada ese día)
    #Sumamos estos conteos a lo largo de todos los días para obtner el total de actores
    # Calculamos el total de actor-dias
    total_actor_days = 0
    day = 0
    for day_scenes in days:
        actors = set()
        for scene in day_scenes:
            actors |= set(np.where(sessions[scene] == 1)[0])
        total_actor_days += len(actors)
        day +=1
    return total_actor_days


    

In [7]:
total_actor_days = 99999
days=[]

print("Forzando a las 6 escenas por día")
for t in range(100):# Una pequeña mejora: buscamos la mejor opción en X intentos, en este caso 100 porque fue el número que me dio 27 la mayoria de veces
    days_n = scenes_organization_gh(sessions, False)
    total_actor_days_n= calc_cost(days_n, sessions)
    if total_actor_days_n < total_actor_days:
        total_actor_days = total_actor_days_n
        days = days_n
day = 1
for day_scenes in days:
    print(f'Scenes of days {day}:{day_scenes}')
    day += 1
print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

total_actor_days = 99999
days = []

print("Without forcing 6 scenes per day:")
for t in range (1):
    days_n = scenes_organization_gh(sessions, True)
    total_actor_days_n = calc_cost(days_n, sessions)
    if total_actor_days_n < total_actor_days:
        total_actor_days = total_actor_days_n
        days = days_n
day = 1
for day_scenes in days:
    print(f'Scenes od day {day}:{day_scenes}')
    day += 1
print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

Forzando a las 6 escenas por día
Scenes of days 1:[23, 17, 13, 16, 18, 22]
Scenes of days 2:[5, 6, 8, 12, 26, 27]
Scenes of days 3:[10, 1, 4, 19, 21, 29]
Scenes of days 4:[2, 14, 3, 7, 20, 28]
Scenes of days 5:[11, 9, 15, 24, 25, 0]
Total times I have to pay if these scenes are scheduled (greedy): 28
Without forcing 6 scenes per day:
Scenes od day 1:[22, 13, 7, 8, 1, 0]
Scenes od day 2:[19, 5, 2, 3, 11, 9]
Scenes od day 3:[14, 16, 17, 20, 4, 6]
Scenes od day 4:[12, 15, 18, 21, 10, 23]
Scenes od day 5:[28, 26, 24, 25]
Scenes od day 6:[29]
Scenes od day 7:[27]
Total times I have to pay if these scenes are scheduled (greedy): 44


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta